In [ ]:
import pandas as pd
import numpy as np
import perf
import metrics

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
%matplotlib inline

In [ ]:
0.217744


In [24]:
! g++ -Wall -std=c++11 -O3 src/*.cpp -o ./bin/sgd

In [25]:
!time ./bin/sgd

rmse: (0) 0.947487 0
rmse: (100000) 0.216787 0
shuffling 81741
shuffling 862553
shuffling 441259
shuffling 25572
shuffling 95101
shuffling 475266
shuffling 127646
shuffling 1344560
shuffling 1188103
shuffling 1242593
shuffling 304713
shuffling 1290636
shuffling 278119
shuffling 1106926
shuffling 148566
shuffling 148252
shuffling 98117
shuffling 626438
shuffling 89079
shuffling 1269017
shuffling 929902
shuffling 1266282
shuffling 779201
shuffling 1136749
shuffling 242734
shuffling 1169697
shuffling 872569
shuffling 669882
shuffling 66870
shuffling 13095
shuffling 571916
shuffling 844729
shuffling 947160
shuffling 1152156
shuffling 207273
shuffling 1221177
shuffling 833790
shuffling 117767
shuffling 987870
shuffling 541218
shuffling 182508
shuffling 828361
shuffling 1068455
shuffling 780118
shuffling 167500
shuffling 674962
shuffling 656539
shuffling 397347
shuffling 649794
shuffling 693433
shuffling 1202444
shuffling 311343
shuffling 451700
shuffling 491436
shuffling 1031106
shuffling 7

In [ ]:
df = pd.read_csv("./data/results.csv")
print('mean:', round(np.sum(df.prediction) / np.sum(df.exposure), 6), round(np.sum(df.target) / np.sum(df.exposure), 6))
print('rmse:', round(perf.root_mean_square_error(df.target, df.prediction, df.exposure), 6))
print('gini:', metrics.gini_emblem_fast(df.target, df.prediction, df.exposure))
print('deviance:', np.log(metrics.poisson_deviance(df.target, df.prediction, df.exposure)))
metrics.plot_lift_curve(df.target, df.prediction, df.exposure)

In [ ]:
0.21607

mean: 0.043815 0.044271
rmse: 0.216067
gini: 0.32862359198864044
deviance: 12.322948734266841